In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/NLP\ project/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1MmY0pN1b5xL_C2CijM9ImcFM-UFt4bwr/NLP project


In [3]:
#import packages
import pandas as pd
import numpy as np
import os
import sys 
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, SimpleRNN
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Data Preprocessing

In [4]:
path = "./songs_edit_revised/"
corpus = ''

all_files = os.listdir(path)
for file in all_files:
  with open(os.path.join(path, file)) as f:
    text = f.read()
    corpus+= '\n\n'+ text


In [5]:
components = corpus.split('\n\n')

In [6]:
print(len(components))

288


In [12]:
corpus_list = [string.split('\n') for string in components]
#corpus_list[:10]
for song in corpus_list:
  song.pop(0)

In [13]:
corpus_list[:10]

[[],
 ['Will you make this island',
  'Amazing in all ways',
  'Surprises every corner',
  'Delightful nights and days',
  'Will you take this country',
  'And turn it from a place',
  'To a home that greets you',
  'With smiles on every face',
  'Will you come on this brave journey',
  'Will you help to make it real',
  'Will you write us grand new stories',
  'Songs that everyone will feel'],
 ['So will you swim the current',
  'Will you scale new heights',
  'Will you make it happen',
  'Will you let your dreams take flight',
  'And will you make the difference',
  'Will you seize the day',
  'Will you live each moment',
  'Will you dare to find new ways'],
 ['Will you take this city',
  'And turn it from a place',
  'To a home that greets you',
  'With smiles on every face',
  'Will you come on this brave journey',
  'Will you help to make it real',
  'Will you write us grand new stories',
  'Songs that everyone will feel'],
 ['So will you swim the current',
  'Will you scale new h

In [15]:
df = pd.DataFrame({'song': corpus_list})

df['song'] = df['song'].str.join(', ')
pd.options.display.max_colwidth = 500
df=df.iloc[1:] 
df.head()

,song
1,"Will you make this island, Amazing in all ways, Surprises every corner, Delightful nights and days, Will you take this country, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
2,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
3,"Will you take this city, And turn it from a place, To a home that greets you, With smiles on every face, Will you come on this brave journey, Will you help to make it real, Will you write us grand new stories, Songs that everyone will feel"
4,"So will you swim the current, Will you scale new heights, Will you make it happen, Will you let your dreams take flight, And will you make the difference, Will you seize the day, Will you live each moment, Will you dare to find new ways"
5,"Dare to find, Dare to find, Dare to find, New ways"


Generating lyrics for VERSE

In [16]:
from google.colab import files
src = list(files.upload().values())[0]
open('utilities.py','wb').write(src)
import utilities as U

Saving utilities.py to utilities (3).py


In [21]:
"""
Remove punctuations, decontractions, emojis and URLs
"""

df['song'] = df.apply(lambda row : U.remove_punct(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.remove_url(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.remove_emoji(row['song']), axis=1)
df['song'] = df.apply(lambda row : U.decontraction(row['song']), axis=1)
df['song']=  df.apply(lambda row : row['song'].lower(), axis=1)
df.head()

,song
1,will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
2,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
3,will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel
4,so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways
5,dare to find dare to find dare to find new ways


In [22]:
songs = df['song'].tolist()

In [23]:
songs[:10]

['will you make this island amazing in all ways surprises every corner delightful nights and days will you take this country and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each moment will you dare to find new ways',
 'will you take this city and turn it from a place to a home that greets you with smiles on every face will you come on this brave journey will you help to make it real will you write us grand new stories songs that everyone will feel',
 'so will you swim the current will you scale new heights will you make it happen will you let your dreams take flight and will you make the difference will you seize the day will you live each

In [24]:
# Save verses in .txt file
with open('songs.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in songs:
        filehandle.write('%s\n' % listitem)

In [25]:
textFileName = 'songs.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()

Word-Level Lyrics Generation

In [26]:
"""
Generate song.
Reference: https://predictivehacks.com/word-based-text-generation/
"""
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
import keras.utils as ku 

tokenizer = Tokenizer()

corpus = raw_text.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
 
label = ku.np_utils.to_categorical(label, num_classes=total_words)
 
 
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Dropout(0.2))
model.add(SimpleRNN(128))
#model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 97, 100)           107000    
                                                                 
 dropout (Dropout)           (None, 97, 100)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               29312     
                                                                 
 dense (Dense)               (None, 535)               69015     
                                                                 
 dense_1 (Dense)             (None, 1070)              573520    
                                                                 
Total params: 778,847
Trainable params: 778,847
Non-trainable params: 0
_________________________________________________________________
None


In [48]:
'''
Neutral input seeds:
Whenever I think back 
And so this I know 
'''

seed_text = "whenever i think back"
next_words = 100

def generateLyrics(seed_text, model):
  next_words = 100   
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
      predicted=np.argmax(model.predict(token_list, verbose=0) ,axis=1)

      output_word = ""
      for word, index in tokenizer.word_index.items():
          if index == predicted:
              output_word = word
              # break
      seed_text += " " + output_word
  return seed_text

lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['whenever i think back well contribute uneventfullest renewed renewed things beat beat been beat yo find oh drop stars renewed nothing because renewed sunsets things renewed things beat so spirits things bravely nothing top echoing been top big stop words beat and peers things test oh proud islands growing spirits stars please doors and mankind realize should once joy granted beat stars things things uneventfullest stars wall joy cross stars well beat stars thought however tomorrows foundations imagine here so conviction eiffel granted action cos stars things worthwhile with stood kid beat granted join anew stronger take seize because youthful guess peers beat four']

In [44]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("whenever i think back.txt", header=False, index=False, encoding="utf-8")

from google.colab import files
files.download("whenever i think back.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
seed_text = "and so this I know"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['and so this I know worthwhile uneventfullest things neighbourhood beat renewed joy working beat things beat beat things things things thing storms                                                                                   ']

In [51]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("and so this I know.txt", header=False, index=False, encoding="utf-8")
files.download("and so this I know.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [52]:
'''
Negative input seeds:
I am tired of being what you want me to be 
Feeling so faithless, lost under the surface 
'''

seed_text = "i am tired of being what you want me to be"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['i am tired of being what you want me to be big experienced beat beat real am crimson taken thing renewed renewed wonderful renewed afar reach things honour open youve been somewhere birth reach afar nothing faith top stars spain thing will stronger tapestry course breathe very guess seen very prepared reach big guess colours spirits spirits spirits very spirits part granted once wildest granted so and comfort real not and strive shine changes i beat grateful spirits comfort future tomorrows looking live hawker in aheads lights beat grew hey guess stars join million youve plays rise again tomorrows beat recognition stars country choose spirits lights paradise been take grain youthful']

In [53]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("i am tired of being what you want me to be.txt", header=False, index=False, encoding="utf-8")
files.download("i am tired of being what you want me to be.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
seed_text = "feeling so faithless lost under the surface"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['feeling so faithless lost under the surface things yo hawker hawker renewed beat beat things                                                                                            ']

In [55]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("feeling so faithless lost under the surface.txt", header=False, index=False, encoding="utf-8")
files.download("feeling so faithless lost under the surface.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [56]:
'''
Positive input seeds:
Relight our fire, we will find our way 
We will rise stronger together  
'''
seed_text = "relight our fire we will find our way"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['relight our fire we will find our way renewed beat stars beat things things things things things around stars stars am things man worthwhile lights open take so renewed to renewed spirits closest bay once beauty belongs uneventfullest beat senses join stars tapestry reach remain foundations very very stars conquer doors paradise i so looking wall hawker seize nothing tomorrows spirits granted spirits stars sun spirits granted fair contribute goal and will spirits stars so unfurled stars granted reach looking fills spirits so stronger run thought is stood real trip thought stood learn reach seven stars run spirits stars hawker place defending open live so matter joy and']

In [57]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("relight our fire we will find our way.txt", header=False, index=False, encoding="utf-8")
files.download("relight our fire we will find our way.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [58]:
seed_text = "we will rise stronger together"
lyrics_list=[]
lyrics = generateLyrics(seed_text, model)
lyrics_list.append(lyrics)
lyrics_list

['we will rise stronger together beat future well to back beat renewed collyer beat rise take tomorrows at top taken selfexplanatory families rise action renewed live itll big youve beat so stood reach tomorrows changes big beat growing stars ours walked breathe seek poor tomorrows grateful things we days walked stop poor hey and quiet fills spirits youthful hawker streams nurtured not at paradise wonderful so brand because bay taken living and will should should horizon page therell well journey so contribute tomorrows top so oh beat leap brand religion hawker and so strive spirits stars stars away at lions man reach experienced changes beat']

In [59]:
#Save Results to text file
outputdf=pd.DataFrame(lyrics_list)
outputdf.to_csv("we will rise stronger together.txt", header=False, index=False, encoding="utf-8")
files.download("we will rise stronger together.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>